In [1]:
import os,re
import requests as rq
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import wikipedia as wiki
import musicbrainzngs as mbz
import spotipy

In [2]:
data = rq.get("https://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&titles=List_of_hip_hop_musicians").json()
list_markup = data["query"]["pages"]["140853"]["revisions"][0]["*"]
link_regex = r"\* \[\[(.+?)[\(\]]"

rappers = re.findall(link_regex, list_markup)

In [3]:
import spotipy.oauth2 as oauth2

# When an error occurs saying the token expired, re-run this cell.
# Tokens are 1 hour I think.
client_id = 'e3344de9f0454df280cc34cdba040bf7'
client_secret = 'da09e980b2fb4f11a68a3c00a2669a7d'

credentials = oauth2.SpotifyClientCredentials(
        client_id=client_id,
        client_secret=client_secret)

token = credentials.get_access_token()
spotify = spotipy.Spotify(auth=token)

In [25]:
def get_albums(rapper):
    results = spotify.search(q=f'artist:{rapper}',type='artist')
    
    artist_id = results['artists']['items'][0]['uri']
    
    albums = spotify.artist_albums(artist_id)
    
    # For some reason duplicates some albums. Wrap as a set
    return set([album['name'] for album in albums['items']])

In [51]:
def get_album_artists(album_name):
    results = spotify.search(q=f'album:{album_name}',type='album')
    
    album_id = results['albums']['items'][0]['uri']
    
    tracks = spotify.album_tracks(album_id)
    return set([artist['name']  
            for track in tracks['items']
            for artist in track['artists']])

In [52]:
def get_collaborators(rapper):
    """Returns a dictionary of album:artists entry"""
    
    albums = get_albums(rapper)
    
    return {album:list(get_album_artists(album)) for album in albums}

In [57]:
get_collaborators("Drake")

{'Nothing Was The Same (Deluxe)': ['2 Chainz',
  'JAY Z',
  'Big Sean',
  'Drake',
  'Majid Jordan',
  'Detail',
  'Jhene Aiko',
  'Sampha'],
 'Views': ['Pimp C',
  'Drake',
  'PARTYNEXTDOOR',
  'WizKid',
  'Kyla',
  'Rihanna',
  'dvsn',
  'Future'],
 "If You're Reading This It's Too Late": ['Drake',
  'PARTYNEXTDOOR',
  'Lil Wayne',
  'Travis Scott'],
 'More Life': ['2 Chainz',
  'Young Thug',
  'Black Coffee',
  'Jorja Smith',
  'Giggs',
  'Quavo',
  'Kanye West',
  'Drake',
  'Travis Scott',
  'PARTYNEXTDOOR',
  'Sampha'],
 'What A Time To Be Alive': ['Drake', 'Future'],
 'Scorpion': ['JAY Z',
  'Ty Dolla $ign',
  'Michael Jackson',
  'Drake',
  'Static Major'],
 'Nothing Was The Same': ['2 Chainz',
  'JAY Z',
  'Big Sean',
  'Drake',
  'Majid Jordan',
  'Detail',
  'Jhene Aiko',
  'Sampha']}